In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import math

In [2]:
# model parameters

Ka = 1e-5
c_bufs = {'Tris':1.1e-3,'TrisH': 8.9e-3}
zs = {'K': 1,'Cl': -1,'H': 1}#,'OH': -1}
Ps = {'K': 0.3e-9,'Cl': 1e-11,'H': 1e-5}#,'OH': 1e-5} #m^s-1
c = 1e-2 #Fm^-2

F = 96485 #Cmol^-1
R = 8.31 #JK^-1mol^-1
r_v = 12.5e-6 #m
T = 298 #K
A = 4*np.pi*r_v**2
vol = 4*np.pi*r_v**3/3
c_outs = {'K':1,'Cl':1,'H':1e-4}#,'OH':1e-4} #Molm^-3


B = np.log(10)*20*Ka*1e-4/(2e-4)**2 #buffer capacity for my buffer. TODO: measure this!
C = c*A
eta = (zs['H']**2)*(F**2)*vol*B/(R*T*np.log(10)*C)
# eta = 1e6

pHo = -1*math.log(c_outs['H'])/(math.log(10))
    
fluxes = {key:[] for key in Ps.keys()}
dCs = {key:[] for key in Ps.keys()}
threshold = 0.05

c_ins = {key:[] for key in Ps.keys()}

V = []

dt = 0.01 #s


# Old model: seemed to be unstable

def initialise():
    
    V.append(0)
    c_ins['K'].append(0)
    c_ins['Cl'].append(0)
    c_ins['H'].append(1e-4)
#     c_ins['OH'].append(1e-4)

    

def dCdt(c_in,c_out,P,V, z):
    
    mu = z*F*V/(R*T)
#     print('mu = ', mu)
#     print(abs(mu))
#     print('mu: ',abs(mu))
    if abs(mu) < threshold:
        
        return (P/(3*r_v))*(c_out - c_in)
    
#     print('mu = ', mu)
    
    prefactor = mu*P/(3*r_v)
    
#     print('exp: ',np.exp(-mu))
    return prefactor*(c_out - c_in*np.exp(mu))/(np.exp(mu)-1)


def dV(dCs,vol,eta,C,zs):
    
    # args:
    # dCs: dictionary of ion molar fluxes
    # vol: volume of GUV
    # eta: dimensionless constant. the voltage change due to one unit of pH change
    # C: capacitance of GUV membrane
    
    prefactor = 1/C
    
    total_Q_flux =0
    for key in dCs.keys():

            
        total_Q_flux += zs[key]*F*vol*dCs[key]
        
#     print(total_Q_flux)
#     print('total charge flux: ',total_Q_flux)
    return prefactor*total_Q_flux


def adjustcH(Ka,c_bufs):

    
    return Ka*c_bufs['TrisH']/c_bufs['Tris']


"""
def calcV(Q,C,A):
#     print(Q)
    return Q/(C*A)


def Q_in(c_ins,vol):
    
    Q = 0 
    for key in c_ins.keys():
#         print(c_ins[key])
        Q += vol*F*zs[key]*c_ins[key][-1]
        
#     print('Q = ',Q)
    return Q
"""    

# def cH(V,pHo):
    

    
#     neg_pH = F*V[-1]/(R*T*math.log(10))-pHo
# #     print('pH inside GUV: ',-neg_pH)
#     return 10**neg_pH

def step_forward(c_ins,V,c_bufs):
    
    # calculate fluxes and increment the internal concentrations
    
    
    for key in c_ins.keys():

            
        dC = dt*dCdt(c_ins[key][-1],c_outs[key],Ps[key],V[-1],zs[key])
        fluxes[key].append(dC/dt)
        dCs[key] = dC
        
        
        c_ins[key].append(c_ins[key][-1]+dC)
        c_ins['H'][-1] = adjustcH(Ka,c_bufs)
        
    # adjust the H+ concentration due to the buffer
    
#     dcH = c_ins['H'][-2] - c_ins['H'][-1]
    
#     c_ins['H'][-1] += adjustcH(c_ins['H'][-1],Ka,c_bufs['-'][-1],dcH)
    
    
#     Q = Q_in(c_ins,vol)
#     print(Q)

    
    V.append(V[-1]+dV(dCs,vol,eta,C,zs))
#     print('V: ',V[-1])



In [78]:
eta

783.1790780123946

In [ ]:
#define times

ts = np.arange(0,30000000)

# initialise


initialise()


# start time loop
counter = 0
for t in ts:
    
    if not t % 1000 and t > 0:
        counter += 1
        lastV = V[-1]
        V = V[:counter]+[lastV]
#         print(V)
        for key in c_ins.keys():
            lastC = c_ins[key][-1]
            c_ins[key] = c_ins[key][:counter]+[lastC]
            
            lastflux = fluxes[key][-1]
            fluxes[key] = fluxes[key][:counter]+[lastflux]
            
    if not t% 10000:
        print(t, ' steps completed')
    
    step_forward(c_ins,V,c_bufs)

0  steps completed
10000  steps completed
20000  steps completed
30000  steps completed
40000  steps completed
50000  steps completed
60000  steps completed
70000  steps completed
80000  steps completed
90000  steps completed
100000  steps completed
110000  steps completed
120000  steps completed
130000  steps completed
140000  steps completed
150000  steps completed
160000  steps completed
170000  steps completed
180000  steps completed
190000  steps completed
200000  steps completed
210000  steps completed
220000  steps completed
230000  steps completed
240000  steps completed
250000  steps completed
260000  steps completed
270000  steps completed
280000  steps completed
290000  steps completed
300000  steps completed
310000  steps completed
320000  steps completed
330000  steps completed
340000  steps completed
350000  steps completed
360000  steps completed
370000  steps completed
380000  steps completed
390000  steps completed
400000  steps completed
410000  steps completed
420000

3340000  steps completed
3350000  steps completed
3360000  steps completed
3370000  steps completed
3380000  steps completed
3390000  steps completed
3400000  steps completed
3410000  steps completed
3420000  steps completed
3430000  steps completed
3440000  steps completed
3450000  steps completed
3460000  steps completed
3470000  steps completed
3480000  steps completed
3490000  steps completed
3500000  steps completed
3510000  steps completed
3520000  steps completed
3530000  steps completed
3540000  steps completed
3550000  steps completed
3560000  steps completed
3570000  steps completed
3580000  steps completed
3590000  steps completed
3600000  steps completed
3610000  steps completed
3620000  steps completed
3630000  steps completed
3640000  steps completed
3650000  steps completed
3660000  steps completed
3670000  steps completed
3680000  steps completed
3690000  steps completed
3700000  steps completed
3710000  steps completed
3720000  steps completed
3730000  steps completed


6630000  steps completed
6640000  steps completed
6650000  steps completed
6660000  steps completed
6670000  steps completed
6680000  steps completed
6690000  steps completed
6700000  steps completed
6710000  steps completed
6720000  steps completed
6730000  steps completed
6740000  steps completed
6750000  steps completed
6760000  steps completed
6770000  steps completed
6780000  steps completed
6790000  steps completed
6800000  steps completed
6810000  steps completed
6820000  steps completed
6830000  steps completed
6840000  steps completed
6850000  steps completed
6860000  steps completed
6870000  steps completed
6880000  steps completed
6890000  steps completed
6900000  steps completed
6910000  steps completed
6920000  steps completed
6930000  steps completed
6940000  steps completed
6950000  steps completed
6960000  steps completed
6970000  steps completed
6980000  steps completed
6990000  steps completed
7000000  steps completed
7010000  steps completed
7020000  steps completed


9910000  steps completed
9920000  steps completed
9930000  steps completed
9940000  steps completed
9950000  steps completed
9960000  steps completed
9970000  steps completed
9980000  steps completed
9990000  steps completed
10000000  steps completed
10010000  steps completed
10020000  steps completed
10030000  steps completed
10040000  steps completed
10050000  steps completed
10060000  steps completed
10070000  steps completed
10080000  steps completed
10090000  steps completed
10100000  steps completed
10110000  steps completed
10120000  steps completed
10130000  steps completed
10140000  steps completed
10150000  steps completed
10160000  steps completed
10170000  steps completed
10180000  steps completed
10190000  steps completed
10200000  steps completed
10210000  steps completed
10220000  steps completed
10230000  steps completed
10240000  steps completed
10250000  steps completed
10260000  steps completed
10270000  steps completed
10280000  steps completed
10290000  steps compl

13080000  steps completed
13090000  steps completed
13100000  steps completed
13110000  steps completed
13120000  steps completed
13130000  steps completed
13140000  steps completed
13150000  steps completed
13160000  steps completed
13170000  steps completed
13180000  steps completed
13190000  steps completed
13200000  steps completed
13210000  steps completed
13220000  steps completed
13230000  steps completed
13240000  steps completed
13250000  steps completed
13260000  steps completed
13270000  steps completed
13280000  steps completed
13290000  steps completed
13300000  steps completed
13310000  steps completed
13320000  steps completed
13330000  steps completed
13340000  steps completed
13350000  steps completed
13360000  steps completed
13370000  steps completed
13380000  steps completed
13390000  steps completed
13400000  steps completed
13410000  steps completed
13420000  steps completed
13430000  steps completed
13440000  steps completed
13450000  steps completed
13460000  st

16250000  steps completed
16260000  steps completed
16270000  steps completed
16280000  steps completed
16290000  steps completed
16300000  steps completed
16310000  steps completed
16320000  steps completed
16330000  steps completed
16340000  steps completed
16350000  steps completed
16360000  steps completed
16370000  steps completed
16380000  steps completed
16390000  steps completed
16400000  steps completed
16410000  steps completed
16420000  steps completed
16430000  steps completed
16440000  steps completed
16450000  steps completed
16460000  steps completed
16470000  steps completed
16480000  steps completed
16490000  steps completed
16500000  steps completed
16510000  steps completed
16520000  steps completed
16530000  steps completed
16540000  steps completed
16550000  steps completed
16560000  steps completed
16570000  steps completed
16580000  steps completed
16590000  steps completed
16600000  steps completed
16610000  steps completed
16620000  steps completed
16630000  st

In [4]:
%matplotlib qt
fig,axs = plt.subplots(1,5)

# t_plot = dt*ts
for i,key in enumerate(c_ins.keys()):

        #
        #
    axs[i].plot(c_ins[key][:-1])
    
    axs[i].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    axs[i].set_title(key,fontsize = 18)
    axs[i].set_xlabel('T [mins]')
#     axs[i].set_xticks([0,1000,2000])
#     axs[i].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
#     axs[i].set_xlim([-50,2050])

    
# axs[0].set_ylim([0,1])
# axs[1].set_ylim([0,1])
axs[4].plot(V[:-1])
axs[4].set_title('V_m')
# axs[3].set_xticks([0,1000,2000])
# axs[3].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
# axs[3].set_xlim([-50,2050])
# axs[3].set_ylim([0,1])

Text(0.5, 1.0, 'V_m')

In [5]:
# %matplotlib qt
fig,axs = plt.subplots(1,4)

for i,key in enumerate(fluxes.keys()):
    
    axs[i].scatter(1-np.array(c_ins[key]),fluxes[key])
    
    axs[i].set_title(key)

(6000, 6000)

In [10]:
# equilibrium ion number in GUV
N = vol #moles
N = N*(6e23) #ions

In [8]:
init_flux = fluxes['K'][0]/100 #molsm^-2s^-1
init_transport_rate = init_flux*A #molss^-1
init_transport_rate = (6e23)*init_transport_rate


In [11]:
N/init_transport_rate

0.5208333333333336